In [7]:
from torch.autograd import Variable
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

class TNet(nn.Module):
    def __init__(self,F,O):
        super(TNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Linear(F, int(F/2)),
            nn.ReLU(inplace=True),
            nn.Linear(int(F/2),F),
            nn.BatchNorm1d(F),
            nn.ReLU(inplace=True),
            nn.Linear(F, O)
            )
#         self.sfmx = nn.Softmax()
        
    def forward(self, x):
        x = self.feature(x)
#         x = self.sfmx(x)
        return x

import numpy as np
import pandas as pd
import random
import os, shutil, copy, math

file = 'Training_Sample.csv'
df = pd.read_csv(file)
data_input = df.as_matrix()

data = {}
labl = {}
data['train'] = torch.from_numpy(data_input[:-50,:-1]).float()#.unsqueeze(1)
data['valid'] = torch.from_numpy(data_input[-50:,:-1]).float()#.unsqueeze(1)
labl['train'] = torch.from_numpy(data_input[:-50,-1])
labl['valid'] = torch.from_numpy(data_input[-50:,-1])

def lr_scheduler(epoch, optimizer):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    init_lr = 0.01
    lr_decay_epoch = 5
    lr = init_lr * (0.4**(epoch // lr_decay_epoch))
    if epoch % lr_decay_epoch == 0: print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups: param_group['lr'] = lr
    return

PHASE = ['train','valid']
model = TNet(16,1).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.7)
num_epochs = 50
criterion = nn.L1Loss()
for epoch in range(num_epochs):
    for phase in PHASE:
        if phase == 'train':
            lr_scheduler(epoch,optimizer)
            model.train(True)  # Set model to training mode
        else:
            model.train(False)  # Set model to evaluate mode
            
        optimizer.zero_grad()
        
        outputs = model(Variable(data[phase].cuda()))
        preds = outputs.data.cpu()
        loss = criterion(outputs, Variable(labl[phase].cuda().float()))
        if phase == 'train': 
            loss.backward()
            optimizer.step()
            
        running_loss = loss.data[0]
        print('{:5}: Loss: {:.4f}'.format(phase, running_loss))    
        

LR is set to 0.01
train: Loss: 178.2754
valid: Loss: 272.2174
train: Loss: 178.2750
valid: Loss: 272.0581
train: Loss: 178.2743
valid: Loss: 271.9783
train: Loss: 178.2735
valid: Loss: 271.9276
train: Loss: 178.2725
valid: Loss: 271.8921
LR is set to 0.004
train: Loss: 178.2714
valid: Loss: 271.8702
train: Loss: 178.2709
valid: Loss: 271.8526
train: Loss: 178.2705
valid: Loss: 271.8375
train: Loss: 178.2700
valid: Loss: 271.8249
train: Loss: 178.2695
valid: Loss: 271.8138
LR is set to 0.0016000000000000003
train: Loss: 178.2690
valid: Loss: 271.8065
train: Loss: 178.2688
valid: Loss: 271.8005
train: Loss: 178.2686
valid: Loss: 271.7954
train: Loss: 178.2684
valid: Loss: 271.7914
train: Loss: 178.2682
valid: Loss: 271.7881
LR is set to 0.0006400000000000002
train: Loss: 178.2680
valid: Loss: 271.7858
train: Loss: 178.2679
valid: Loss: 271.7841
train: Loss: 178.2678
valid: Loss: 271.7826
train: Loss: 178.2677
valid: Loss: 271.7816
train: Loss: 178.2677
valid: Loss: 271.7806
LR is set to 